# Import packages

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import mean_squared_error, mean_absolute_error, \
    r2_score, make_scorer, recall_score, accuracy_score, f1_score, \
    precision_score, balanced_accuracy_score, roc_curve, auc

# Custom functions

In [ ]:
from pickle_managment import save_pickle, load_pickle

# Regression

In [ ]:
regression_df_expanded_cleaned = pd.read_csv(
    r'datasets\train_datasets\regression_df_expanded_cleaned_train.csv.zip'
)
regression_df_expanded_cleaned

In [ ]:
regression_X = regression_df_expanded_cleaned.loc[
               :,
               ~regression_df_expanded_cleaned.columns.isin(
                   ['SMILES', 'logBB'])
               ]

regression_y = regression_df_expanded_cleaned['logBB']

# Use the transformer that the SVM regressor has used before
data_processing_pipeline =load_pickle(r'model_outputs\svm_regressor\svm_regressor_pipeline.pkl')
regression_X_processed=data_processing_pipeline.transform(regression_X)
regression_X_processed

In [ ]:
regression_X_train, regression_X_test, regression_y_train, regression_y_test = train_test_split(
    regression_X_processed,
    regression_y,
    test_size=0.2,
    random_state=1,
    shuffle=True
)

In [ ]:
save_pickle(regression_X_train,
            r'model_outputs\rf_regressor\regression_X_train.pkl')

save_pickle(regression_y_train,
            r'model_outputs\rf_regressor\regression_y_train.pkl')

save_pickle(regression_X_test,
            r'model_outputs\rf_regressor\regression_X_test.pkl')

save_pickle(regression_y_test,
            r'model_outputs\rf_regressor\regression_y_test.pkl')

## Random forest

### Model training

In [ ]:
start_time = datetime.now()

rf_regressor = RandomForestRegressor(
    random_state=1,
    bootstrap=True, #Whether bootstrap samples are used when building trees.
    # Default to be True
    oob_score=True #Whether to use out-of-bag samples to estimate the
    # generalization score. Only available if bootstrap=True
)

rf_regressor_grid_search = GridSearchCV(
    estimator=rf_regressor,
    param_grid={
        'n_estimators': [1000, 3500],  #Number of trees
        'max_depth': [5, 20],  #The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples
        'min_samples_split': [2, 30], #The minimum number of samples required
        # to split an internal node. Default to be 2
        'min_samples_leaf': [5, 30], #The minimum number of samples required
        # to be at a leaf node. The branch will stop splitting once the
        # leaves have that number of samples each. Having 1 here is more
        # prone to overfitting
        # 'criterion': ['squared_error'] # “squared_error” for the mean
        # squared error. This is the default
        'max_samples':[0.2, 0.85],
        'max_features': ['sqrt', 'log2'] #Reduce the number of features can
        # mitigate overfitting issue
    },
    cv=8,  #Number of fold for cross validation. It should be 8 or 10
    scoring={
        # All these are only viable in the negative option
        'MAE': 'neg_mean_absolute_error',
        'MSE': 'neg_mean_squared_error',
        'R2': 'r2'
    },
    refit='R2', #MAE is less sensitive to outliers and can help reduce
    # overfitting

    n_jobs=1,
    # -1 means using all processors, but it won't give you any messages.
    # Only using 1 for my computer print out the training messages

    verbose=10  #Provide detailed more messages
)

rf_regressor_grid_search.fit(regression_X_train, regression_y_train)

end_time = datetime.now()
print('GridSearchCV took {}'.format(end_time - start_time))

In [ ]:
rf_regressor_results_df = pd.DataFrame(rf_regressor_grid_search.cv_results_)
#Make the GridSearch results into a df
rf_regressor_results_df.drop(
    list(rf_regressor_results_df.filter(regex='time|split|std')),
    axis=1,
    inplace=True
)  # Remove columns that aren't very interesting

rf_regressor_results_df = rf_regressor_results_df.sort_values(
    by='rank_test_R2')
rf_regressor_results_df

In [ ]:
rf_regressor_results_df.to_csv(
    r'model_grid_search\rf_regressor_results.csv',
    index=False
)

In [ ]:
best_rf_regressor = rf_regressor_grid_search.best_estimator_
save_pickle(
    best_rf_regressor,
    r'model_pickles\best_rf_regressor.pkl'
)
# To load this best model again, use load_pickle(r'model_pickles\random_forest_regressor\best_rf_regressor.pkl')

# Classification

## Neural network
According to SVM classifiers' performance using the 2 methods of balancing
the dataset, the one created by centroid gave more reliable better results.
Consequently, that dataset is used to train the MLP classifier

In [ ]:
classification_df_expanded_balanced = pd.read_csv(
    r'datasets\balanced_datasets\BBB_classification_balanced_centroid.csv.zip'
)
classification_df_expanded_balanced

In [ ]:
# Data pre-processing. This will make the model less interpretable but since
# MLP is already less interpretable than RF, this is fine since I will try
# to use another RF to explain why MLP made such decision

classification_X = classification_df_expanded_balanced.loc[
                   :,
                   ~classification_df_expanded_balanced.columns.isin(
                       ['SMILES', 'BBB+/BBB-'])
                   ]

classification_y = classification_df_expanded_balanced['BBB+/BBB-']


data_processing_pipeline =load_pickle\
    (r'model_outputs\svm_classifier\centroid_pipeline.pkl')

classification_X_processed=data_processing_pipeline.transform(classification_X)
classification_X_processed

In [ ]:
classification_X_train, classification_X_test, classification_y_train, classification_y_test = train_test_split(
    classification_X_processed,
    classification_y,
    test_size=0.2,
    random_state=1,
    shuffle=True,
    stratify=classification_y #Ensure train set and test set have the same
    # ratio for the 2 categories
)

In [ ]:
save_pickle(classification_X_train,
            r'model_outputs\mlp_classifier\classification_X_train.pkl')

save_pickle(classification_y_train,
            r'model_outputs\mlp_classifier\classification_y_train.pkl')

save_pickle(classification_X_test,
            r'model_outputs\mlp_classifier\classification_X_test.pkl')

save_pickle(classification_y_test,
            r'model_outputs\mlp_classifier\classification_y_test.pkl')

### Model training

In [ ]:
start_time = datetime.now()

mlp_classifier = MLPClassifier(
    solver='adam',
    #"adam" is the default. It works pretty well on relatively large datasets (with thousands of training samples or more) in terms of both training time and validation score
    random_state=1,
    shuffle=True,  #shuffle samples in each iteration for "adam" solver
)

mlp_classifier_grid_search = GridSearchCV(
    estimator=mlp_classifier,
    param_grid={
        'hidden_layer_sizes': [(10,),(50,),(10,10)],
        'alpha': [1e-3, 1], #1e-4 is the default. Need stronger
        # regularization to avoid overfitting
        'activation': [
            'relu', #"relu" is the default
            'tanh'
        ],
        'batch_size': [32, 128], #Smaller batch size can help reducing
        # overfitting. Since the dataset has feature number > 200, the
        # default is to use 200 as the batch_size
        'learning_rate': ['constant', 'adaptive'] #Default to be "constant"
    },
    cv=RepeatedStratifiedKFold(
        n_splits=8,
        n_repeats=2, #Each time the split will be different
        random_state=1
    ),
    scoring={
        'Recall': make_scorer(
            recall_score, #Need pos_label
            pos_label='BBB+', #Without this, pos_label is default to be 1
            # and will through an error since 1 isn't "BBB+" or "BBB-"
            average='binary'
        ),
        'Precision': make_scorer(
            precision_score, #Need pos_label
            pos_label='BBB+',
            average='binary'
        ),
        'F1': make_scorer(
            f1_score, #Need pos_label
            pos_label='BBB+',
            average='binary'
        ),
        'Accuracy': 'accuracy', #accuracy_score doesn't need pos_label
        'Balanced accuracy': 'balanced_accuracy',
        'AUROC': 'roc_auc'
    },
    refit='AUROC',

    n_jobs=1,
    verbose=10
)

mlp_classifier_grid_search.fit(classification_X_train, classification_y_train)

end_time = datetime.now()
print('GridSearchCV took {}'.format(end_time - start_time))

In [ ]:
mlp_classifier_results_df = pd.DataFrame(mlp_classifier_grid_search.cv_results_)
#Make the GridSearch results into a df

mlp_classifier_results_df.drop(
    list(mlp_classifier_results_df.filter(regex='time|split|std')),
    axis=1,
    inplace=True
)  # Remove columns that aren't very interesting
mlp_classifier_results_df = mlp_classifier_results_df.sort_values(
    by='rank_test_AUROC')

mlp_classifier_results_df

In [ ]:
mlp_classifier_results_df.to_csv(
    r'model_grid_search\mlp_classifier_results.csv',
    index=False
)

In [ ]:
best_mlp_classifier = mlp_classifier_grid_search.best_estimator_
save_pickle(
    best_mlp_classifier,
    r'model_pickles\best_mlp_classifier.pkl'
)
# To load this best model again, use load_pickle
# (r'model_pickles\mlp_classifier\best_mlp_classifier.pkl')